# Runtime test notebook

In [1]:
import rewardlm

## Data Utils

In [2]:
from rewardlm.data.data_utils import gen_benchmark_data
from rewardlm.core.GenerativeModel import GenerativeModel

generative_manager = GenerativeModel(model_id = 'EleutherAI/pythia-70m')

gen_benchmark_data(generative_manager.tokenizer)

Using MPS (Apple Silicon GPU)


Found cached dataset json (/Users/daniel/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


## Reward model

In [3]:
from rewardlm.core.RewardModel import RewardModel
reward_manager = RewardModel(model_id = 'facebook/roberta-hate-speech-dynabench-r4-target', device='mps')

### Reward tokenizer

In [6]:
reward_manager.tokenize_text('Hello, world!')

{'input_ids': tensor([[    0, 31414,     6,   232,   328,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [10]:
# Trying w/ batch text
reward_manager.tokenize_text(
    ['First sentence', 'second longer sentence'],
    padding = 'max_length', 
    max_length = 12, 
    truncation = True,
)

{'input_ids': tensor([[    0, 10993,  3645,     2,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 10815,  1181,  3645,     2,     1,     1,     1,     1,     1,
             1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

### Model itself

In [7]:
reward_manager.get_score_pair(
    prompt = reward_manager.tokenize_text('This is a prompt'),
    response = reward_manager.tokenize_text('This is the continuation of the prompt')
)

(array([0.00119799], dtype=float32), array([0.00028152], dtype=float32))

In [13]:
from torch.utils.data import DataLoader, Dataset
from rewardlm.data.CustomDatasets import ToxicityGeneratedSet
import pandas as pd

# trying w/ batch text
batch_input = reward_manager.tokenize_text(
    ['First sentence', 'second longer sentence', 'third one'],
    padding = 'max_length', 
    max_length = 12, 
    truncation = True,
)

dataset = ToxicityGeneratedSet(pd.DataFrame)
        
reward_manager.get_batch_score_pair(
    DataLoader(batch_input, batch_size=1)
)

TypeError: __init__() missing 1 required positional argument: 'tokenizer'

## RL model

In [2]:
from rewardlm.core.RL.RLModel import RLModel

# sample data (thanks chatGPT for the help)
sample_data = [
    "I love to eat pizza.",
    "The cat is sleeping.",
    "She plays the guitar.",
    "He is a good friend.",
    "The sun is shining.",
    "They went to the park.",
    "I enjoy reading books.",
    "We love going to movies.",
    "The dog is barking.",
    "She loves to dance.",
    "He is a great chef.",
    "The flowers are blooming.",
    "They won the game.",
    "I need a cup of coffee.",
    "She sings beautifully.",
    "He likes to swim.",
    "The rain is pouring.",
    "They are best friends.",
    "I like to go hiking.",
    "She runs every morning.",
]

print('new rl')
rlmanager = RLModel(
    model_id = 'EleutherAI/pythia-70m',
    reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target',
    optimized = False,
    device = 'cpu',
    bs = 20,     # TMP TODO, Remove it when done tesing on sample data (defaults to 256)
)
custom_prompt = 'Someone said {prompt}'
print('Generating dataset')
dataset = rlmanager.generate_dataset(
    text = sample_data,
    max_len = 50,
    custom_prompt = custom_prompt,
)

assert rlmanager.generator_manager.tokenizer.decode(dataset[0]['input_ids'], skip_special_tokens = True) == custom_prompt.format(prompt = sample_data[0]), 'Something wrong'
print('Dataset done')

history = rlmanager.train_PPO(dataset = dataset)

new rl


/Users/daniel/miniforge3/envs/torch_new/lib/python3.9/site-packages/trl/trainer/ppo_trainer.py:221: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


Generating dataset
Dataset done
mps
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


0it [00:00, ?it/s]

Epoch 1:


100%|██████████| 1/1 [00:00<00:00,  1.88it/s]
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
0it [00:11, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!